# Green OA Dataset Builder Complete

Will build dataframe of
- ORCID
- DOI
- Policy Finder info

for
- journal articles
- constrained from a start date to today
- for an arbitrary ORCID search, defaults to `current-institution-affiliation-name:("Brock University")`

Will need
- ORCID API key and secret
- Policy Finder API secret


Saves as CSV file.  

Project code in [repository](https://github.com/elibtronic/green_oa_compiler/).


In [4]:
#Libraries etc
# Will ask you to restart the session!
!pip install orcid
!pip install crossrefapi

import requests
import json
import orcid
import pprint
import pandas as pd

from crossref.restful import Works
from google.colab import files
from datetime import datetime
from urllib.parse import urlparse

pd.set_option('display.max_rows', None)

print("\nLibraries installed and loaded, Ready to proceed!")


Libraries installed and loaded, Ready to proceed!


In [8]:
# @title # Parameters {"vertical-output":true,"display-mode":"form"}

# @markdown ## ORCID API Settings

Client_ID = "" # @param {"type":"string"}
Client_Secret = "" # @param {"type":"string"}

# @markdown ## ORCID Search Settings

s_query = "current-institution-affiliation-name:(\"Brock University\")"#@param {type:"string"}

# @markdown ---

# @markdown ## CrossRef Search Settings

# @markdown Date to start DOI harvest from?
Start_Date = "2025-01-01" # @param {"type":"date"}
s_date = datetime.strptime(Start_Date, '%Y-%m-%d')

# @markdown Date to end DOI harvest from? **Not yet implemeted**
End_Date = "" # @param {"type":"date"}
e_date = datetime.strptime(End_Date, '%Y-%m-%d')

# @markdown ---

# @markdown ## Policy Finder API Settings

pf_key = "" # @param {"type":"string"}

# @markdown ---

# @markdown ## ORCID List Settings


# @markdown Fetch pre-compiled ORCID list from the following URL if not building from scratch
orcid_list_url = "https://github.com/elibtronic/green_oa_compiler/raw/refs/heads/main/ORCID_List.csv" # @param {"type":"string"}

# @markdown Refresh list of ORCIDs? (will do add / drop info as well)
refresh_orcid = True # @param {"type":"boolean","placeholder":"True"}

# @markdown ORCIDs to add to fetched results
orcid_add_list_url = "https://raw.githubusercontent.com/elibtronic/green_oa_compiler/refs/heads/main/ORCID_add.csv" # @param {"type":"string"}

# @markdown ORCIDs to drop from list
orcid_drop_list_url = "https://raw.githubusercontent.com/elibtronic/green_oa_compiler/refs/heads/main/ORCID_drop.csv" # @param {"type":"string"}

api = orcid.PublicAPI(Client_ID, Client_Secret, sandbox=False)
search_token = api.get_search_token_from_orcid()

print("Options set and API connections completed!")

Options set and API connections completed!


## ORCID List Creation

Can build from scratch using search string if `refresh_orcid` is checked above or will pull pre-existing ORCID list from Repository

In [ ]:
b_members = []

if refresh_orcid == True:
  #Build a dictionary of ORCIDs currently affiliated with Brock
  #Also need to check verified email that is with brocku.ca
  search_results = api.search_generator(s_query, access_token=search_token)

  print("Retrieving Fresh set ORCIDs\n")
  prog = 0
  for page in search_results:
    b_members.append(page['orcid-identifier']['path'])

    #some low-fi progress feedback
    prog += 1
    if prog % 10 == 0:
      print(str(prog)+"..", end = "")
    if prog % 100 == 0:
      print("\n")


  print("\nProcessing Add/Drop Lists.\n")
  #Add list
  add_members_df = pd.read_csv(orcid_add_list_url)
  add_members = add_members_df['ORCID'].tolist()
  b_members += add_members

  #Drop list
  drop_members_df = pd.read_csv(orcid_drop_list_url)
  drop_members = drop_members_df['ORCID'].tolist()

  for drop_member in drop_members:
    if drop_member in b_members:
      b_members.remove(drop_member)

  #Will save new list
  b_members_df = pd.DataFrame(b_members, columns = ['ORCID'])
  b_members_df.to_csv('ORCID_List.csv',index=False)
  files.download('ORCID_List.csv')

else:
  print("Loading ORCID list from: ",orcid_list_url)
  b_members_df = pd.read_csv(orcid_list_url)
  b_members = b_members_df['ORCID'].tolist()

print("\nORCID API list constructed, ", len(b_members), " entries found")

## DOI retrieval

Will now attempt to retrieve _journal articles_ with author _ORCID_ found in the list constructed above. Begins with `Start_Date` defined above. Also standarizes License URLs

(This will take some time to run. Easily 20 or so minutes to get 900-ish records.)

In [ ]:
harvested_j_works = []
works = Works()
#check for most recent pub date of items associated with that orcid
#extract some metadata for that, doi, title, journal, etc.
#created a df with orcid of person and doi etc

print("Retrieving works from ",len(b_members)," ORCID records\n")
prog = 0
for b_orcid in b_members:

  #some low-fi progress feedback
  prog += 1
  if prog % 50 == 0:
    print(str(prog)+"..", end = "")
  if prog % 500 == 0:
    print("\n")

  works_list = api.read_record_public(b_orcid, 'works',search_token)
  for w in works_list['group']:

    date_work = datetime.fromtimestamp(w['last-modified-date']['value'] / 1000)
    if date_work >= s_date:
      for ws in w.get("work-summary"):
        if ws.get('type') == 'JOURNAL_ARTICLE':
          eids = ws.get('external-ids')
          for e in eids['external-id']:

            if e.get('external-id-type') == 'doi':
              doi =  e.get('external-id-value')
              item = works.doi(doi)
              try:
                license_url = item['license'][0]['URL']
              except:
                license_url = "No URL retrieved"

              try:
                issn = item['ISSN'][0]
              except:
                issn = "No ISSN found"

              harvested_j_works.append([b_orcid,doi,issn,license_url])

print("\n\nDone harvesting DOIs from CrossRef.")

#Turn retrieved info into Dataframe
df = pd.DataFrame(harvested_j_works)
df.columns = ['ORCID',"DOI","issn","LICENSE_URL"]
df.drop_duplicates(subset=['DOI'],inplace=True)
df.reset_index(drop=True,inplace=True)


#Standardizing URLs from DOI
standard_url = []
for key, value in df.iterrows():

  try:
    standard_url.append(urlparse(value['LICENSE_URL']).netloc)
  except:
    standard_url.append("No URL retrieved")

df['standard_url'] = standard_url
#this would be the spot to drop LICENSE_URL if it doesn't help
print("\nDone standardizing URLs.")


print("\nReady to proceed.")

## Policy Finder Data

Will now augment DOI set with info retrieved from The Policy Finder API.

This will also take a long time to run.

In [ ]:
#As there is no Python wrapper for this API, I'll make a function
#that will use requests to grab the JSON
def pf_fetch(pf_key,issn):
  '''Returns JSON from Policy Finder given an ISSN and API Key'''
  paip_options = {
      "item-type" : "publication",
      "api-key" : pf_key,
      "format" : "Json",
  }

  paip_options['identifier'] = issn

  requests.packages.urllib3.disable_warnings()
  api_blob = requests.get("https://v2.sherpa.ac.uk/cgi/retrieve_by_id",params=paip_options,verify=False).json()

  if api_blob['items'] == []:
    return None
  else:
    return api_blob['items'][0]

In [ ]:
list_uri = []
list_submitted_oa_policy = []
list_accepted_oa_policy = []
list_published_oa_policy = []
prog = 0

print("Retrieving PolicyFinder data for ", len(df), " DOIs")

for key, value in df.iterrows():

  #some low-fi progress feedback
  prog += 1
  if prog % 50 == 0:
    print(str(prog)+"..", end = "")
  if prog % 500 == 0:
    print("\n")

  if value['issn'] == "No ISSN found":
    #print("no issn")
    list_uri.append("None")
    list_submitted_oa_policy.append("None")
    list_accepted_oa_policy.append("None")
    list_published_oa_policy.append("None")
  else:
    #print(value['issn'])
    par = pf_fetch(pf_key,value['issn'])

    #No policy found in otherwords
    if par == None:
      list_uri.append("None")
      list_submitted_oa_policy.append("None")
      list_accepted_oa_policy.append("None")
      list_published_oa_policy.append("None")
    else:
      policy_uri = par['system_metadata']['uri']
      accepted_oa_policy = ""
      published_oa_policy = ""
      submitted_oa_policy = ""
      for pub_policy in par['publisher_policy']:
        for per_oa in pub_policy['permitted_oa']:
          try:
            #Only keep ones that have IR as option, for first pass
            if 'institutional_repository' in per_oa['location']['location']:
              #accepted policy first
              if per_oa['article_version'][0] == 'accepted':
                try:
                  conditions_text = " ".join(per_oa['conditions'])
                except:
                  conditions_text = "no conditions"
                if per_oa['additional_oa_fee'] == "no":
                  fee_text = "fee_no"
                else:
                  fee_text = "fee_yes"
                per_id = str(per_oa['id'])
                accepted_oa_policy += per_id+":"+fee_text+":"+conditions_text+";"

              #published
              elif per_oa['article_version'][0] == 'published':
                try:
                  conditions_text = " ".join(per_oa['conditions'])
                except:
                  conditions_text = "no conditions"
                if per_oa['additional_oa_fee'] == "no":
                  fee_text = "fee_no"
                else:
                  fee_text = "fee_yes"
                per_id = str(per_oa['id']).strip()
                published_oa_policy += per_id+":"+fee_text+":"+conditions_text+";"

              #submitted
              elif per_oa['article_version'][0] == 'submitted':
                try:
                  conditions_text = " ".join(per_oa['conditions'])
                except:
                  conditions_text = "no conditions"
                if per_oa['additional_oa_fee'] == "no":
                  fee_text = "fee_no"
                else:
                  fee_text = "fee_yes"
                per_id = str(per_oa['id']).strip()
                submitted_oa_policy += per_id+":"+fee_text+":"+conditions_text+";"
          except: #on occasion it error out totally
            #print("error")
            policy_uri = "error"
            accepted_oa_policy = "error"
            published_oa_policy = "error"
            submitted_oa_policy = "error"

        if submitted_oa_policy == "":
          submitted_oa_policy = "None"
        if accepted_oa_policy == "":
          accepted_oa_policy = "None"
        if published_oa_policy == "":
          published_oa_policy = "None"

      #print("\n")
      #print(policy_uri)
      list_uri.append(policy_uri)
      #print(submitted_oa_policy)
      list_submitted_oa_policy.append(submitted_oa_policy)
      #print(accepted_oa_policy)
      list_accepted_oa_policy.append(accepted_oa_policy)
      #print(published_oa_policy)
      list_published_oa_policy.append(published_oa_policy)

# add these new columns to the df
df['policy_uri'] = list_uri
df['submitted_oa_policy'] = list_submitted_oa_policy
df['accepted_oa_policy'] = list_accepted_oa_policy
df['published_oa_policy'] = list_published_oa_policy

print("\nDone retrieving PolicyFinder data.")

In [ ]:
#Let's have a look
df.sample(10)

,ORCID,DOI,issn,LICENSE_URL,standard_url,policy_uri,submitted_oa_policy,accepted_oa_policy,published_oa_policy
786,0009-0007-6904-959X,10.3138/CTR.150.3,0315-0836,No URL retrieved,,https://v2.sherpa.ac.uk/id/publication/6686,None,354:fee_no:Must link to published article with...,None
77,0000-0002-1577-2658,10.1016/j.jbc.2022.102568,0021-9258,https://www.elsevier.com/tdm/userlicense/1.0/,www.elsevier.com,https://v2.sherpa.ac.uk/id/publication/10347,None,None,None
546,0000-0003-1476-2965,10.1177/097135570701600202,0971-3557,https://journals.sagepub.com/page/policies/tex...,journals.sagepub.com,https://v2.sherpa.ac.uk/id/publication/9520,None,None,None
604,0000-0001-6203-5338,10.1002/adfm.202000594,1616-301X,http://onlinelibrary.wiley.com/termsAndConditi...,onlinelibrary.wiley.com,https://v2.sherpa.ac.uk/id/publication/228,11204:fee_no:Must acknowledge acceptance for p...,None,None
127,0000-0001-8925-3108,10.18848/2327-7955/CGP/v32i02/67-88,2327-7955,No URL retrieved,,https://v2.sherpa.ac.uk/id/publication/31146,423:fee_no:Must acknowledge publisher copyrigh...,8291:fee_no:Published source must be acknowled...,None
229,0000-0002-9613-4323,10.1016/j.jcorpfin.2021.101965,0929-1199,https://www.elsevier.com/tdm/userlicense/1.0/,www.elsevier.com,https://v2.sherpa.ac.uk/id/publication/13836,None,11179:fee_no:Must link to publisher version wi...,11181:fee_yes:Published source must be acknowl...
930,0000-0002-5449-3352,10.1139/h11-050,1715-5312,http://www.nrcresearchpress.com/page/about/Cor...,www.nrcresearchpress.com,https://v2.sherpa.ac.uk/id/publication/10122,8195:fee_no:no conditions;,3875:fee_no:Publisher copyright and source mus...,None
922,0000-0002-5449-3352,10.1139/h2012-042,1715-5312,http://www.nrcresearchpress.com/page/about/Cor...,www.nrcresearchpress.com,https://v2.sherpa.ac.uk/id/publication/10122,8195:fee_no:no conditions;,3875:fee_no:Publisher copyright and source mus...,None
364,0000-0003-1476-2965,10.1177/00218863211059185,0021-8863,https://creativecommons.org/licenses/by/4.0/,creativecommons.org,https://v2.sherpa.ac.uk/id/publication/9516,None,None,None
174,0009-0004-6000-473X,10.1080/01916122.2025.2500055,0191-6122,No URL retrieved,,https://v2.sherpa.ac.uk/id/publication/5919,None,None,None


## Export Data

Final CSV file of the compiled data!

In [ ]:
#DOI data as CSV
csv_file_name = "Brock_ORCID_Harvest_Start_Date_"+str(Start_Date)+"_to_"+datetime.today().strftime('%Y-%m-%d')+".csv"
print("Saving to file: "+csv_file_name)
df.to_csv(csv_file_name,index=False)
files.download(csv_file_name)

Saving to file: Brock_ORCID_Harvest_Start_Date_2025-01-01_to_2025-08-26.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>